In [ ]:
!rm -rf ./ML4CV_Assignment
!git clone https://github.com/liuktc/ML4CV_Assignment.git

import sys
sys.path.append('/kaggle/working/ML4CV_Assignment')

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import json

import torch
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
import torch.nn as nn
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import numpy as np

from plot import color, semantic_embeddings_plot
from dataset import StreetHazardDataset, PadToMultipleOf14
from _model import DINOv2_SemanticSegmentation
from loss import nt_xent_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

COLAB = False
KAGGLE = False
NUM_CLASSES = 13
SEED = 42

# Seed everything
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
if COLAB:
    !wget https://people.eecs.berkeley.edu/~hendrycks/streethazards_train.tar -O train.tar
    !tar -xf train.tar

## Data Preparation


In [ ]:
ORIGINAL_IMAGE_SIZE = (720, 1280)
IMAGE_SIZE = PadToMultipleOf14().convert_dims((480,480))
BATCH_SIZE = 1
POSITIVE_PAIRS = True
PIXEL_PER_CLASS = 3

# Percorsi dei file
if COLAB:
    annotations_file = "/content/train/train.odgt"
    img_dir = "/content/train"
elif KAGGLE:
    annotations_file = "../input/streethazards-train/train/train.odgt"
    img_dir = "../input/streethazards-train/train"
else:
    annotations_file = "./data/train/train.odgt"
    img_dir = "./data/train"


image_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE, interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
    ]
)

target_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE, interpolation=InterpolationMode.NEAREST),
    ]
)


# Crea il dataset
dataset = StreetHazardDataset(
    annotations_file,
    img_dir,
    image_transform=image_transform,
    target_transform=target_transform,
    positive_pairs=POSITIVE_PAIRS,
    pixel_per_class=PIXEL_PER_CLASS
)

val_perc = 0.2
indices = torch.randperm(len(dataset)).tolist()

val_dataset = Subset(dataset, indices[:int(len(dataset) * val_perc)])
train_dataset = Subset(dataset, indices[int(len(dataset) * val_perc):])

dl_val = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
dl_train = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# Show a sample
for image, segmentation, (selected_pixels ,target_matrix) in dl_train:
        
    print(f"Image shape: {image[0].shape}")
    print(f"Segmentation shape: {segmentation[0].shape}")
    print(f"Selected pixels: {selected_pixels[0]}")
    print(segmentation[0].flatten()[selected_pixels[0]])
    print(f"Target matrix: {target_matrix[0][0]}")
    plt.imshow(color(segmentation[0].cpu().numpy()))
    plt.show()
    break

# Semantic Segmentation

In [ ]:
from train import train_semantic_segmentation

backbone = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14", pretrained=True)

criterion = nn.CrossEntropyLoss()
EPOCHS = 10
LOAD_WEIGHTS = False
LEARNING_RATE = 1e-3
EMBEDDING_SIZE = 256

segmentation_model = DINOv2_SemanticSegmentation(
    num_classes=NUM_CLASSES, image_size=IMAGE_SIZE, embedding_size=EMBEDDING_SIZE
).to(device)

for param in segmentation_model.feature_extractor.parameters():
    param.requires_grad = False

In [ ]:
if LOAD_WEIGHTS:
    if COLAB:
        path_prefix = "/content/"
    elif KAGGLE:
        path_prefix = "/kaggle/input/"
    else:
        path_prefix = "./"
    path = os.path.join(path_prefix, "segmentation_model.pth")
    segmentation_model.load_state_dict(torch.load(path, map_location=device))

In [ ]:
segmentation_optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, segmentation_model.parameters()), lr=LEARNING_RATE
)

train_semantic_segmentation(
    model=segmentation_model,
    criterion=criterion,
    dl_train=dl_train,
    dl_val=dl_val,
    device=device,
    epochs=EPOCHS,
    optimizer=segmentation_optimizer,
    original_image_size=ORIGINAL_IMAGE_SIZE
)

In [ ]:
# Save the segmentation model
torch.save(segmentation_model.state_dict(), "segmentation_model.pth")

## Metric Learning


In [ ]:
x = torch.tensor([[[1,1,-5,-5], [1,1,-5,-5], [1,1,-5,-5]]])
# (B, num_pixels, emb_size)
selected_pixels = torch.tensor([[0, 1, 2, 3]])

target_matrix = torch.tensor([[[1, 1, 0, 0],
                              [1, 1, 0, 0],
                              [0, 0, 1, 1],
                              [0, 0, 1, 1]]])
print("x.shape",x.shape)
print("selected_pixels.shape", selected_pixels.shape)
print("target_matrix.shape", target_matrix.shape)
loss = nt_xent_loss(x, selected_pixels, target_matrix=target_matrix, temperature=0.1)
print(loss)

In [ ]:
EPOCHS = 10
TEMPERATURE = 0.1
LEARNING_RATE = 5e-4
LOAD_WEIGHTS = False
EMBEDDING_SIZE = 256

metric_learning_model = DINOv2_SemanticSegmentation(
    num_classes=None, image_size=IMAGE_SIZE, embedding_size=EMBEDDING_SIZE
).to(device)

if LOAD_WEIGHTS:
    if COLAB:
        path_prefix = "/content/drive/"
    elif KAGGLE:
        path_prefix = "/kaggle/working/"
    else:
        path_prefix = "./"
    path = os.path.join(path_prefix, "metric_learning_model2.pth")

    metric_learning_model.load_state_dict(torch.load(path, map_location=device))

for param in metric_learning_model.feature_extractor.parameters():
    param.requires_grad = False
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, metric_learning_model.parameters()),
    lr=LEARNING_RATE,
)


for epoch in range(EPOCHS):
    for index, (images, _, (selected_pixels, target_matrix)) in tqdm(enumerate(dl_train)):
        images = images.to(device)
        
        # Compute the pixel embeddings        
        pixel_embeddings = metric_learning_model(images).to(device)

        # Compute the metric learning loss
        loss = nt_xent_loss(
            pixel_embeddings, selected_pixels, target_matrix, TEMPERATURE, device=device
        )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}/{EPOCHS} - {loss.item()}")

        if index % 200 == 0:
            semantic_embeddings_plot(metric_learning_model, dl_train, num_points=1000)

In [ ]:
if COLAB:
    !wget https://people.eecs.berkeley.edu/~hendrycks/streethazards_test.tar -O test.tar
    !tar -xf test.tar

In [ ]:
# Load test dataset
if COLAB:
    annotations_file = "/content/test/test.odgt"
    img_dir = "/content/test"
elif KAGGLE:
    annotations_file = "/kaggle/input/test/test.odgt"
    img_dir = "/kaggle/input/test"
else:
    annotations_file = "./data/test/test.odgt"
    img_dir = "./data/test"

test_dataset = StreetHazardDataset(
    annotations_file,
    img_dir,
    image_transform=image_transform,
    target_transform=target_transform,
    positive_pairs=True,
    pixel_per_class=40
)

dl_test = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
for images, segmentations, (pos_pairs, pos_pairs_relative) in dl_test:
    print(images.size(), segmentations.size(), pos_pairs.size())
    
    segmentations = segmentations[0].cpu().numpy()
    pos_pairs = pos_pairs[0].cpu().numpy()
    
    plt.imshow(color(segmentations))
    break
    
    # print(segmentations.shape, pos_pairs.shape)
    
    # for i in pos_pairs:
    #     print(segmentations[i[0]], segmentations[i[1]])
    # break

In [ ]:
semantic_embeddings_plot(metric_learning_model, dl_test, num_points=100000)

In [ ]:
import torch, gc
import os

gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Save the model
# torch.save(metric_learning_model.state_dict(), "model.pth")

# metric_learning_model = torch.load("./model2.pth", map_location=torch.device('cpu'))
# Load the state dict model
metric_learning_model = DINOv2_SemanticSegmentation(num_classes=None, image_size=IMAGE_SIZE, fine_grained_features_size=128)
metric_learning_model.load_state_dict(torch.load("model2.pth", map_location=torch.device('cpu')))

## Idea for outlier detection

- For each segmentation class, calculate its distribution (Gaussian assumption, so just calculate mean and variance).
- With this pre-calculated distributions, we can calculate, for each pixel, the probability of it belonging to a certain class. If the $\max$ of these probabilities is low, we can classify that pixel as outlier (rely on threshold).

In [ ]:
from tqdm.auto import tqdm
def calculate_distribution_for_each_class(metric_learning_model, dl, num:int = 100):
    res = {}
    count = 0
    for images, segmentations, _ in tqdm(dl, total=num // 4):
        count += len(images)
        images = images.to(device)
        segmentations = segmentations.to(device)
        unique_classes = torch.unique(segmentations)
        
        # Calculate the embedding for each pixel
        pixel_embeddings = metric_learning_model(images)
        
        B = len(images)
        for i in range(B):
            # For each class, calculate the mean of the embeddings
            for c in unique_classes:
                mask = segmentations[i] == c
                
                # print("mask", mask.shape)
                # print("pixel_embeddings", pixel_embeddings[i].shape)
                # print("images", images[i].shape)
                # print("segmentations", segmentations[i].shape)
                c = c.item()
                if mask.sum() > 1:
                    if c not in res:
                        # print("AAAAAA", pixel_embeddings[i][:, mask].shape)
                        res[c] = {"mean": pixel_embeddings[i][:, mask].mean(dim=(1)).cpu().detach().numpy(),
                                "std": pixel_embeddings[i][:, mask].std(dim=(1)).cpu().detach().numpy(),
                                "count": mask.sum().cpu().detach().numpy(),}
                        
                        # print("res[c]['mean']", res[c]["mean"].shape)
                        # print("res[c]['std']", res[c]["std"].shape)
                        # print("res[c]['count']", res[c]["count"])
                    else:
                        res[c]["mean"] = (res[c]["mean"] * res[c]["count"] + pixel_embeddings[i][:, mask].sum(dim=(1)).cpu().detach().numpy()) / (res[c]["count"] + mask.sum().cpu().detach().numpy())
                        res[c]["std"] = (res[c]["std"] * res[c]["count"] + pixel_embeddings[i][:, mask].std(dim=(1)).cpu().detach().numpy()) / (res[c]["count"] + mask.sum().cpu().detach().numpy())
                        # res[c][]
                        res[c]["count"] += mask.sum().cpu().detach().numpy()
        print(count)
        if count > num:
            break
    return res

def multivariate_pdf(x, means, cov_matrix):
    d = len(means)
    return np.exp(-0.5 * (x - means).T @ np.linalg.inv(cov_matrix) @ (x - means)) / np.sqrt(np.linalg.det(cov_matrix) * (2 * np.pi) ** d)

# class_std_inv_cache = {}
def probability_belong_to_class(pixel_embedding, class_mean, class_std):
    # if tuple(class_std) not in class_std_inv_cache:
    #     class_std_inv_cache[tuple(class_std)] = np.linalg.inv(np.diag(class_std))
    #     # class_std_inv_cache[tuple(class_std)] = np.diag(class_std)
        
        
    # class_std_inv = class_std_inv_cache[tuple(class_std)]
    # print("x-m",(pixel_embedding - class_mean))
    # print("class_std_inv", class_std_inv)
    # print("prod", (pixel_embedding - class_mean).T @ class_std_inv)
    # print("final prod", (pixel_embedding - class_mean).T @ class_std_inv @ (pixel_embedding - class_mean))
    # print((pixel_embedding - class_mean).T @ class_std_inv @ (pixel_embedding - class_mean))
    # return 1 / np.exp(np.linalg.norm((pixel_embedding - class_mean) * class_std))
    # print(pixel_embedding.shape)
    # print(class_mean.shape)
    return F.cosine_similarity(torch.tensor(pixel_embedding), torch.tensor(class_mean), dim=0)
    # return np.exp(-1 * np.linalg.norm((pixel_embedding - class_mean) / (class_std * 640)))
    # return multivariate_pdf(pixel_embedding, class_mean, class_std) / multivariate_pdf(class_mean, class_mean, class_std)

def classify_new_pixel(pixel_embedding, class_distributions):
    res = {}
    for c in class_distributions:
        m = class_distributions[c]["mean"]
        s = class_distributions[c]["std"]
        res[c] = probability_belong_to_class(pixel_embedding, m, s)
    return res   

In [ ]:
import numpy as np
distributions = calculate_distribution_for_each_class(metric_learning_model, dl_train, num=25)

In [ ]:
distributions[1]

In [ ]:
# For each class, print the probability distribution of the pixel belonging to that class (expected to be close to 1)
results = {}
for images, segmentations, _ in dl_train:
    images = images.to(device)
    segmentations = segmentations.to(device)
    pixel_embeddings = metric_learning_model(images)
    pixel_embeddings = pixel_embeddings.view(pixel_embeddings.size(0),pixel_embeddings.size(1), -1)
    segmentations = segmentations.view(segmentations.size(0), -1)
    
    B = len(images)
    for i in range(B):
        # print(pixel_embeddings[i].shape, pixel_embeddings[i].size(0))
        # pixel_embeddings[i] = pixel_embeddings[i].view(pixel_embeddings[i].size(0), -1)
        for c in tqdm(torch.unique(segmentations[i])):
            mask = segmentations[i] == c
            mask_len = mask.sum()
            masked_pixel_embeddings = pixel_embeddings[i][:, mask]
            # for j in tqdm(range(mask_len)):
            for j in tqdm(range(min(100, mask_len))):
                p = masked_pixel_embeddings[:, j].detach().cpu().numpy()
                # print(p.shape)
                # prob = classify_new_pixel(p, distributions)
                # print(p)
                # print(distributions[c.item()]["mean"])
                # if c.item() not in results:
                #     results[c.item()] = {}
                # print(np.linalg.norm((p - distributions[c.item()]["mean"]) / distributions[c.item()]["std"]) / 640)
                prob = probability_belong_to_class(p, distributions[c.item()]["mean"], distributions[c.item()]["std"])
                # Calculate the distance between the pixel embedding and the mean of each class (considering the variance)
                # for other_c in torch.unique(segmentations[i]):
                #     if other_c.item() not in results[c.item()]:
                #         results[c.item()][other_c.item()] = []
                #     other_c = other_c.item()
                #     prob = np.linalg.norm((p - distributions[other_c]["mean"]) / np.sqrt(distributions[other_c]["std"])) / 640
                #     results[c.item()][other_c].append(prob)
                # print(prob)
                # break
            # break
        
                if c.item() not in results:
                    results[c.item()] = []
                    
                results[c.item()].append(prob)            
                
            # for j in range(len(images[i, :, mask])):
            #     pixel_embedding = pixel_embeddings[i][:, j]
            #     print(pixel_embedding.shape)
            #     prob = probability_belong_to_class(pixel_embedding,
            #                                        distributions[c.item()]["mean"],
            #                                        np.diag(distributions[c.item()]["std"].detach().cpu().numpy()))
                
            #     if c.item() not in results:
            #         results[c.item()] = []
                
            #     results[c.item()].append(prob)
    break

In [ ]:
for c in results:
    print(f"Class {c}")
    for other_c in results[c]:
        print(f"    Class {other_c} - {np.mean(results[c][other_c])}")

In [ ]:
plt.bar(x=[i for i in distributions.keys()], height=[distributions[i]["std"].mean() for i in distributions.keys()])

In [ ]:
import math
# Plot the distributions
for c in results.keys():
    # if np.sum(np.isnan(results[c])) > 0:
    #     continue
    results[c] = [x for x in results[c] if not math.isnan(x)]
    print(results[c])
    plt.hist(results[c], bins=10, label=f"Class {c}", density=True)
    plt.xlim(-1, 1)
    plt.title(f"Class {c}")
    plt.show()

In [ ]:
a = torch.rand(100, 280, 280)
a = a.view(a.size(0), -1)
print(a.shape)
# print(a.mean(dim=(1, 2)).shape)

In [ ]:
# For each class, print the probability distribution of the pixel belonging to that class (expected to be close to 1)
results = {}
for images, segmentations, _ in dl_test:
    images = images.to(device)
    segmentations = segmentations.to(device)
    pixel_embeddings = metric_learning_model(images)
    pixel_embeddings = pixel_embeddings.view(pixel_embeddings.size(0),pixel_embeddings.size(1), -1)
    segmentations = segmentations.view(segmentations.size(0), -1)
    
    B = len(images)
    for i in range(B):
        # print(pixel_embeddings[i].shape, pixel_embeddings[i].size(0))
        # pixel_embeddings[i] = pixel_embeddings[i].view(pixel_embeddings[i].size(0), -1)
        # Take all the pixels of class 14
        mask = segmentations[i] == 5
        masked_pixel_embeddings = pixel_embeddings[i][:, mask]
        mask_len = mask.sum()
        for j in tqdm(range(mask_len)):
            p = masked_pixel_embeddings[:, j].detach().cpu().numpy()
            prob = classify_new_pixel(p, distributions)
            print(prob.values())
            plt.bar(list(prob.keys()),list(prob.values()), label=f"Class 14")
            # Print entropy of the values
            print(-1 * np.sum([x * np.log(x) for x in prob.values()]))
            break
        break
            # for j in range(len(images[i, :, mask])):
            #     pixel_embedding = pixel_embeddings[i][:, j]
            #     print(pixel_embedding.shape)
            #     prob = probability_belong_to_class(pixel_embedding,
            #                                        distributions[c.item()]["mean"],
            #                                        np.diag(distributions[c.item()]["std"].detach().cpu().numpy()))
                
            #     if c.item() not in results:
            #         results[c.item()] = []
                
            #     results[c.item()].append(prob)
    break

In [ ]:
distributions[1]["std"] * distributions[1]["mean"]

In [ ]:
import torch
A = torch.rand(2, 4)
B = torch.rand(2, 4)
print(torch.mm(A,B.T))